<a href="https://colab.research.google.com/github/rizabukhari/Agentic-AI-Labs/blob/main/Lab_5_Implementing_Advanced_Prompting_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Lab 5: Implementing Advanced Prompting Techniques**

**Objective:** This lab moves beyond basic prompt construction to powerful techniques that enhance the reasoning, accuracy, and reliability of Large Language Models (LLMs). You will learn and apply **Zero-Shot**, **Few-Shot**, **Chain-of-Thought (CoT)**, and **Tree-of-Thought (ToT)** prompting to solve complex banking problems.

**Tools:**

  * **LangChain**
  * **OpenAI GPT-4o Mini**


</br>


### **Setup**

Ensure your environment is ready. You will need the same setup as Lab 1.

**.env file:**

```
OPENAI_API_KEY="your_api_key_here"
```


In [ ]:
%%capture
!pip install langchain langchain-openai langchain-community langchainhub

In [ ]:
# USe this if running from Google Colab
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
## Load environment variables from local usng .env.
## WARNING: DO NOT RUN THIS CODE IN COLAB !!!
# import os
# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
llm = ChatOpenAI(model="gpt-5-mini")

# Helper function to print results neatly
def execute_prompt(prompt_template, title, **kwargs):
    """
    Executes a given prompt template with context variables and prints the output.
    """
    print(f"--- {title} ---")

    # Create the prompt using the template and input variables
    prompt = ChatPromptTemplate.from_template(prompt_template)

    # Create the chain
    chain = prompt | llm | StrOutputParser()

    # Invoke the chain with the provided context
    response = chain.invoke(kwargs)

    print("PROMPT:")
    print(prompt.format(**kwargs))
    print("\n" + "="*20 + " RESPONSE " + "="*20 + "\n")
    print(response)
    print("\n" + "-"*50 + "\n")

-----

## **1. Zero-Shot Prompting: Instructing Without Examples**

### **Background**

**Zero-shot prompting** is the most fundamental technique. It involves asking the model to perform a task for which it has received no specific, in-prompt examples. The model relies entirely on its pre-existing knowledge to understand the instruction and generate a response.

  * **How it works:** You give a direct command. For example, "Translate this text to French." The model has seen countless translation tasks during its training and knows what to do.
  * **Strengths:** Simple, fast, and effective for straightforward tasks that fall within the model's general capabilities (e.g., summarization, general classification, simple Q\&A).
  * **Weaknesses:** It can struggle with novel, highly specific, or nuanced tasks where the desired output format or logic is not obvious.
  * **Banking Use Cases:** Initial customer feedback analysis, categorizing support tickets into broad topics, drafting simple emails, identifying the general intent of a customer query.

### **Use Case 1.1: Sentiment Analysis of Bank Feedback**

**Scenario:** A bank's automated system collects feedback from a customer after a service call. We need to classify the sentiment.

#### **Naive Prompt 👎**

This prompt is too simple and gives the model no guidance on the output format or classification scheme.

In [ ]:
customer_feedback_1 = "The mobile app is really slow to load my account balance, and it crashed twice yesterday. I'm very frustrated."

naive_prompt_1 = "{feedback}"

execute_prompt(
    naive_prompt_1,
    "1.1 Naive Sentiment Analysis",
    feedback=customer_feedback_1
)

--- 1.1 Naive Sentiment Analysis ---
PROMPT:
Human: The mobile app is really slow to load my account balance, and it crashed twice yesterday. I'm very frustrated.

==================== RESPONSE ====================

I'm sorry to hear that you're experiencing issues with the mobile app. Here are a few steps you can try to improve its performance:

1. **Update the App**: Make sure you have the latest version of the app installed. Updates often include bug fixes and performance improvements.

2. **Clear Cache**: If the app allows it, try clearing the cache. This can help improve loading times.

3. **Restart Your Device**: Sometimes, simply restarting your device can resolve performance issues.

4. **Check Your Internet Connection**: Ensure that you have a stable internet connection, as a weak connection can cause loading delays.

5. **Reinstall the App**: If the problems persist, consider uninstalling and then reinstalling the app.

6. **Contact Support**: If none of these steps work, it 

**Expected Response:**
The response will be a verbose rephrasing of the feedback, not a clean classification.

#### **Improved Zero-Shot Prompt 👍**

This prompt gives a clear **role**, **task**, and **strict output constraints**, guiding the model to perform a classification task accurately.

In [ ]:
customer_feedback_1 = "The mobile app is really slow to load my account balance, and it crashed twice yesterday. I'm very frustrated."

zeroshot_prompt_1 = """
You are a sentiment analysis AI for a major bank. Your task is to classify customer feedback into one of three categories: Positive, Negative, or Neutral.

Analyze the following feedback and provide only the category name as the output.

Feedback: "{feedback}"
"""

execute_prompt(
    zeroshot_prompt_1,
    "1.1 Improved Zero-Shot Sentiment Analysis",
    feedback=customer_feedback_1
)

--- 1.1 Improved Zero-Shot Sentiment Analysis ---
PROMPT:
Human: 
You are a sentiment analysis AI for a major bank. Your task is to classify customer feedback into one of three categories: Positive, Negative, or Neutral.

Analyze the following feedback and provide only the category name as the output.

Feedback: "The mobile app is really slow to load my account balance, and it crashed twice yesterday. I'm very frustrated."


==================== RESPONSE ====================

Negative

--------------------------------------------------



-----

### **Use Case 1.2: Identifying Customer Intent**

**Scenario:** A chatbot needs to understand why a customer is messaging to route them to the correct department.

#### **Naive Prompt 👎**

This prompt is a vague question that will lead to a conversational and unstructured answer.

In [ ]:
customer_query_1 = "Hi, I can't seem to find my last month's credit card statement in the app."

naive_prompt_2 = "What does this customer want to do? Query: {query}"

execute_prompt(
    naive_prompt_2,
    "1.2 Naive Intent Recognition",
    query=customer_query_1
)

--- 1.2 Naive Intent Recognition ---
PROMPT:
Human: What does this customer want to do? Query: Hi, I can't seem to find my last month's credit card statement in the app.

==================== RESPONSE ====================

The customer wants to locate and access their credit card statement from the previous month in the app.

--------------------------------------------------



#### **Improved Zero-Shot Prompt 👍**

This prompt defines a set of valid intents, forcing the model to classify the query into a predefined category suitable for automated routing.

In [ ]:
customer_query_1 = "Hi, I can't seem to find my last month's credit card statement in the app."

zeroshot_prompt_2 = """
You are an intent recognition model for a banking chatbot. Classify the user's query into one of the following predefined categories:
- Card Services
- Account Inquiry
- Loan Application
- Technical Support
- General Question

User Query: "{query}"

Return only the category name.
"""

execute_prompt(
    zeroshot_prompt_2,
    "1.2 Improved Zero-Shot Intent Recognition",
    query=customer_query_1
)

--- 1.2 Improved Zero-Shot Intent Recognition ---
PROMPT:
Human: 
You are an intent recognition model for a banking chatbot. Classify the user's query into one of the following predefined categories:
- Card Services
- Account Inquiry
- Loan Application
- Technical Support
- General Question

User Query: "Hi, I can't seem to find my last month's credit card statement in the app."

Return only the category name.


==================== RESPONSE ====================

Card Services

--------------------------------------------------



-----

## **2. Few-Shot Prompting: Learning from Examples**

### **Background**

**Few-shot prompting** involves providing the LLM with a few examples ("shots") of the task you want it to perform directly within the prompt. This helps the model understand the desired pattern, format, and nuances.

  * **How it works:** You show the model input-output pairs. For instance, `Input: Sea otter -> Output: Enhydra lutris. Input: Manatee -> Output: Trichechus. Input: Blue whale -> Output:`. The model learns the pattern (common name to scientific name) and completes it.
  * **Strengths:** Extremely effective for tasks requiring a specific output structure (like JSON), a particular style, or classification into custom, non-obvious categories.
  * **Weaknesses:** Requires crafting high-quality examples, and makes the prompt longer and more expensive to process.
  * **Banking Use Cases:** Extracting specific data from unstructured documents (KYC, invoices), categorizing transactions into bank-specific buckets, formatting data for API calls, drafting communications that match a brand's voice.

### **Use Case 2.1: Custom Transaction Categorization**

**Scenario:** A bank's personal finance management tool needs to categorize transactions based on its own unique category list.

#### **Naive Prompt 👎**

The model will use its own general knowledge of categories, which won't match the bank's internal system.

In [ ]:
transaction_description_1 = "ZOMATO-FOOD-DELIVERY GURGAON"

naive_prompt_3 = "Categorize this transaction: '{transaction}'"

execute_prompt(
    naive_prompt_3,
    "2.1 Naive Transaction Categorization",
    transaction=transaction_description_1
)

--- 2.1 Naive Transaction Categorization ---
PROMPT:
Human: Categorize this transaction: 'ZOMATO-FOOD-DELIVERY GURGAON'

==================== RESPONSE ====================

The transaction 'ZOMATO-FOOD-DELIVERY GURGAON' can be categorized as a **Food & Dining** expense, specifically under **Food Delivery Services**.

--------------------------------------------------



This is a good guess, but it may not match our internal category `Food & Entertainment`.

#### **Few-Shot Prompt 👍**

By providing examples, we teach the model our exact categorization schema.

In [ ]:
transaction_description_1 = "ZOMATO-FOOD-DELIVERY GURGAON"

fewshot_prompt_1 = """
You are a transaction categorization engine for a bank. Your goal is to classify transactions into one of the following categories: 'Utilities', 'Groceries', 'Travel', 'Food & Entertainment', 'Shopping', 'Miscellaneous'.

Here are some examples:
1. Transaction: "AMAZON-PAY-ELECTRICITY"
   Category: Utilities
2. Transaction: "UBER-TRIP-MUMBAI-AIRPORT"
   Category: Travel
3. Transaction: "BIGBASKET-ONLINE-GROC"
   Category: Groceries

Now, categorize the following transaction. Provide only the category name.

Transaction: "{transaction}"
Category:
"""

execute_prompt(
    fewshot_prompt_1,
    "2.1 Few-Shot Transaction Categorization",
    transaction=transaction_description_1
)

--- 2.1 Few-Shot Transaction Categorization ---
PROMPT:
Human: 
You are a transaction categorization engine for a bank. Your goal is to classify transactions into one of the following categories: 'Utilities', 'Groceries', 'Travel', 'Food & Entertainment', 'Shopping', 'Miscellaneous'.

Here are some examples:
1. Transaction: "AMAZON-PAY-ELECTRICITY"
   Category: Utilities
2. Transaction: "UBER-TRIP-MUMBAI-AIRPORT"
   Category: Travel
3. Transaction: "BIGBASKET-ONLINE-GROC"
   Category: Groceries

Now, categorize the following transaction. Provide only the category name.

Transaction: "ZOMATO-FOOD-DELIVERY GURGAON"
Category:


==================== RESPONSE ====================

Food & Entertainment

--------------------------------------------------



### **Use Case 2.2: Extracting Structured Data from Text**

**Scenario:** A wealth manager receives an email from a potential client. We need to extract key details into a structured JSON object for the CRM system.

#### **Naive Prompt 👎**

This will produce an unpredictable summary, not a clean, machine-readable object.

In [ ]:
client_email_1 = "Hi, my name is Rohan Gupta. I'm 42 years old and I'm interested in your wealth management services. My investment portfolio is currently around $500,000 and my risk tolerance is moderately aggressive. You can reach me at rohan.gupta@email.com."

naive_prompt_4 = "Extract the client details from this email: {email}"

execute_prompt(
    naive_prompt_4,
    "2.2 Naive Data Extraction",
    email=client_email_1
)

--- 2.2 Naive Data Extraction ---
PROMPT:
Human: Extract the client details from this email: Hi, my name is Rohan Gupta. I'm 42 years old and I'm interested in your wealth management services. My investment portfolio is currently around $500,000 and my risk tolerance is moderately aggressive. You can reach me at rohan.gupta@email.com.

==================== RESPONSE ====================

Client Details:

- Name: Rohan Gupta
- Age: 42
- Interest: Wealth management services
- Investment Portfolio: $500,000
- Risk Tolerance: Moderately aggressive
- Email: rohan.gupta@email.com

--------------------------------------------------



#### **Few-Shot Prompt 👍**

The example teaches the model the exact JSON schema we require.

In [ ]:
client_email_1 = "Hi, my name is Rohan Gupta. I'm 42 years old and I'm interested in your wealth management services. My investment portfolio is currently around $500,000 and my risk tolerance is moderately aggressive. You can reach me at rohan.gupta@email.com."

fewshot_prompt_2 = """
You are a data extraction AI that converts unstructured client emails into structured JSON objects.

---
Example 1:
Email: "My name is Priya Singh, my risk appetite is conservative and my portfolio is $1.2M. I am 55."
JSON:
{{
  "name": "Priya Singh",
  "age": 55,
  "portfolio_value": 1200000,
  "risk_profile": "Conservative",
  "contact_email": "N/A"
}}
---

Now, process the following email and provide only the JSON object as output.

Email: "{email}"
JSON:
"""

execute_prompt(
    fewshot_prompt_2,
    "2.2 Few-Shot Data Extraction",
    email=client_email_1
)

--- 2.2 Few-Shot Data Extraction ---
PROMPT:
Human: 
You are a data extraction AI that converts unstructured client emails into structured JSON objects.

---
Example 1:
Email: "My name is Priya Singh, my risk appetite is conservative and my portfolio is $1.2M. I am 55."
JSON:
{
  "name": "Priya Singh",
  "age": 55,
  "portfolio_value": 1200000,
  "risk_profile": "Conservative",
  "contact_email": "N/A"
}
---

Now, process the following email and provide only the JSON object as output.

Email: "Hi, my name is Rohan Gupta. I'm 42 years old and I'm interested in your wealth management services. My investment portfolio is currently around $500,000 and my risk tolerance is moderately aggressive. You can reach me at rohan.gupta@email.com."
JSON:


==================== RESPONSE ====================

{
  "name": "Rohan Gupta",
  "age": 42,
  "portfolio_value": 500000,
  "risk_profile": "Moderately Aggressive",
  "contact_email": "rohan.gupta@email.com"
}

--------------------------------------

-----

## **3. Chain-of-Thought (CoT) Prompting: Showing Your Work**

### **Background**

**Chain-of-Thought (CoT) prompting** is a technique that encourages the model to break down a complex problem into intermediate steps before giving a final answer. By simply adding phrases like "Let's think step by step," you can significantly improve performance on tasks that require logic, reasoning, or calculation.

  * **How it works:** Instead of jumping to a conclusion, the model first "thinks" out loud, detailing its reasoning process. This mimics how humans solve complex problems and often leads the model to a more accurate result.
  * **Strengths:** Massively improves accuracy on arithmetic, commonsense, and symbolic reasoning tasks. The output is more transparent and auditable, as you can see *how* the model reached its conclusion.
  * **Banking Use Cases:** Assessing loan eligibility based on multiple criteria, preliminary fraud analysis, calculating financial ratios, explaining complex financial products to customers, and checking compliance with internal policies.

### **Use Case 3.1: Preliminary Fraud Alert Analysis**

**Scenario:** A transaction is flagged for potential fraud. A junior analyst needs a summary explaining why it's suspicious.

#### **Naive Prompt 👎**

This prompt asks for a simple "yes/no" answer, which is not helpful and lacks any justification.

In [ ]:
transaction_data_1 = """
- Customer: Anjali Rao
- Usual Location: Bangalore, IN
- Transaction Amount: ₹85,000
- Transaction Location: Prague, CZ
- Time: 3:15 AM IST
- Merchant Category: Luxury Watches
- Customer History: No prior international transactions. Average transaction value is ₹2,500.
"""

naive_prompt_5 = "Is this transaction likely fraudulent? Transaction Details: {transaction}"

execute_prompt(
    naive_prompt_5,
    "3.1 Naive Fraud Analysis",
    transaction=transaction_data_1
)

--- 3.1 Naive Fraud Analysis ---
PROMPT:
Human: Is this transaction likely fraudulent? Transaction Details: 
- Customer: Anjali Rao
- Usual Location: Bangalore, IN
- Transaction Amount: ₹85,000
- Transaction Location: Prague, CZ
- Time: 3:15 AM IST
- Merchant Category: Luxury Watches
- Customer History: No prior international transactions. Average transaction value is ₹2,500.


==================== RESPONSE ====================

Based on the transaction details provided, several factors suggest that this transaction may be likely fraudulent:

1. **Location Discrepancy**: The customer, Anjali Rao, is usually located in Bangalore, India, but the transaction is taking place in Prague, Czech Republic. This significant geographical difference raises a red flag, especially since the customer has no prior international transactions.

2. **Transaction Amount**: The transaction amount of ₹85,000 is substantially higher than the customer's average transaction value of ₹2,500. This large deviatio

#### **Chain-of-Thought Prompt 👍**

We explicitly ask the model to reason through the evidence step-by-step, providing a clear, auditable analysis.

In [ ]:
transaction_data_1 = """
- Customer: Anjali Rao
- Usual Location: Bangalore, IN
- Transaction Amount: ₹85,000
- Transaction Location: Prague, CZ
- Time: 3:15 AM IST
- Merchant Category: Luxury Watches
- Customer History: No prior international transactions. Average transaction value is ₹2,500.
"""

cot_prompt_1 = """
You are a fraud detection analyst. Analyze the following transaction details to determine if it is likely fraudulent.
Let's think step by step to build your analysis before making a final conclusion.

Transaction Details:
{transaction}

First, identify all the risk factors. For each factor, explain why it is suspicious.
Second, summarize your findings.
Finally, conclude with a risk assessment: LOW, MEDIUM, or HIGH.
"""

execute_prompt(
    cot_prompt_1,
    "3.1 Chain-of-Thought Fraud Analysis",
    transaction=transaction_data_1
)

--- 3.1 Chain-of-Thought Fraud Analysis ---
PROMPT:
Human: 
You are a fraud detection analyst. Analyze the following transaction details to determine if it is likely fraudulent.
Let's think step by step to build your analysis before making a final conclusion.

Transaction Details:

- Customer: Anjali Rao
- Usual Location: Bangalore, IN
- Transaction Amount: ₹85,000
- Transaction Location: Prague, CZ
- Time: 3:15 AM IST
- Merchant Category: Luxury Watches
- Customer History: No prior international transactions. Average transaction value is ₹2,500.


First, identify all the risk factors. For each factor, explain why it is suspicious.
Second, summarize your findings.
Finally, conclude with a risk assessment: LOW, MEDIUM, or HIGH.


==================== RESPONSE ====================

### Step 1: Identify Risk Factors

1. **Transaction Location (Prague, CZ) vs. Usual Location (Bangalore, IN)**:
   - **Suspicion**: The transaction is taking place in a foreign country where the customer does 


### **3.2. Chain-of-Thought (CoT): Evaluating a Small Business Loan**

**Scenario:** A loan officer needs a quick, reasoned assessment of a small business loan application to decide if it warrants a full underwriting process.

In [ ]:
loan_application_data = """
- Business Name: 'Global Exports Inc.'
- Years in Business: 2
- Annual Revenue: ₹75,00,000
- Requested Loan Amount: ₹20,00,000
- Owner's CIBIL Score: 680
---
Bank's Minimum Criteria:
- Minimum Years in Business: 3
- Minimum Annual Revenue: ₹50,00,000
- Minimum CIBIL Score: 720
"""

#### **Naive Prompt 👎**

A direct question will yield a "black box" answer with minimal justification, making it hard to trust or audit.

In [ ]:
naive_prompt_3 = "Based on the data and criteria, should we approve a loan for Global Exports Inc.? \n\n{data}"

execute_prompt(
    naive_prompt_3,
    "3.1 Naive Loan Assessment",
    data=loan_application_data
)

--- 3.1 Naive Loan Assessment ---
PROMPT:
Human: Based on the data and criteria, should we approve a loan for Global Exports Inc.? 


- Business Name: 'Global Exports Inc.'
- Years in Business: 2
- Annual Revenue: ₹75,00,000
- Requested Loan Amount: ₹20,00,000
- Owner's CIBIL Score: 680
---
Bank's Minimum Criteria:
- Minimum Years in Business: 3
- Minimum Annual Revenue: ₹50,00,000
- Minimum CIBIL Score: 720


==================== RESPONSE ====================

Based on the provided data and the bank's minimum criteria, here is the assessment for approving a loan for Global Exports Inc.:

1. **Years in Business**: Global Exports Inc. has been in business for 2 years, while the bank's minimum requirement is 3 years. **(Not Met)**

2. **Annual Revenue**: The company has an annual revenue of ₹75,00,000, which exceeds the bank's minimum requirement of ₹50,00,000. **(Met)**

3. **Owner's CIBIL Score**: The owner's CIBIL score is 680, which is below the bank's minimum requirement of 720. **(


*Critique:* The answer is correct, but it lacks a structured breakdown of the decision process.

#### **Improved CoT Prompt 👍**

This prompt forces the model to perform a step-by-step analysis, comparing each application data point against the criteria before reaching a conclusion.

In [ ]:
cot_prompt_3 = """
You are a junior loan underwriter AI. Evaluate the small business loan application against the bank's criteria. Let's think step by step to create a clear evaluation report.

Application Data and Criteria:
{data}

**Evaluation Process:**
1.  **Check Years in Business:** Compare the applicant's years in business against the minimum requirement. State Pass or Fail.
2.  **Check Annual Revenue:** Compare the applicant's revenue against the minimum requirement. State Pass or Fail.
3.  **Check CIBIL Score:** Compare the owner's CIBIL score against the minimum requirement. State Pass or Fail.
4.  **Summary of Findings:** Briefly list the strengths and weaknesses of the application.
5.  **Final Recommendation:** Conclude with 'Proceed to Underwriting' or 'Reject'.
"""

execute_prompt(
    cot_prompt_3,
    "3.2 CoT Loan Assessment",
    data=loan_application_data
)

--- 3.2 CoT Loan Assessment ---
PROMPT:
Human: 
You are a junior loan underwriter AI. Evaluate the small business loan application against the bank's criteria. Let's think step by step to create a clear evaluation report.

Application Data and Criteria:

- Business Name: 'Global Exports Inc.'
- Years in Business: 2
- Annual Revenue: ₹75,00,000
- Requested Loan Amount: ₹20,00,000
- Owner's CIBIL Score: 680
---
Bank's Minimum Criteria:
- Minimum Years in Business: 3
- Minimum Annual Revenue: ₹50,00,000
- Minimum CIBIL Score: 720


**Evaluation Process:**
1.  **Check Years in Business:** Compare the applicant's years in business against the minimum requirement. State Pass or Fail.
2.  **Check Annual Revenue:** Compare the applicant's revenue against the minimum requirement. State Pass or Fail.
3.  **Check CIBIL Score:** Compare the owner's CIBIL score against the minimum requirement. State Pass or Fail.
4.  **Summary of Findings:** Briefly list the strengths and weaknesses of the applicat


*Critique:* The response is transparent, auditable, and provides a clear, well-supported rationale for the decision.


-----

## **4. Tree-of-Thought (ToT) Prompting: Exploring Multiple Paths**

### **Background**

**Tree-of-Thought (ToT)** is an advanced technique that generalizes Chain-of-Thought. Instead of just one linear reasoning path, ToT encourages the model to explore *multiple* different lines of reasoning (branches of a tree). It can then self-evaluate these paths and choose the most promising one to pursue.

  * **How it works:** A full ToT implementation is complex and often requires an agentic framework with multiple LLM calls. However, we can simulate the *spirit* of ToT in a single prompt by asking the model to:
    1.  **Generate** multiple potential solutions or viewpoints.
    2.  **Evaluate** the pros and cons of each.
    3.  **Select** the best option based on that evaluation.
  * **Strengths:** Exceptionally powerful for complex problems with no single right answer, such as strategic planning, creative problem-solving, and generating robust recommendations.
  * **Banking Use Cases:** Developing personalized financial plans for clients, creating customer retention strategies, brainstorming new product features, and formulating responses to complex regulatory inquiries.

### **Use Case 4.1: Proposing a Client Investment Strategy**

**Scenario:** A financial advisor needs to propose an investment strategy for a young client with a moderate risk appetite.

#### **Naive Prompt 👎**

This gives a generic, one-size-fits-all answer without considering alternatives or trade-offs.

In [ ]:
client_profile_1 = """
- Name: Sameer Khan
- Age: 28
- Goal: Wealth accumulation for early retirement
- Risk Tolerance: Moderate
- Annual Income: ₹25,00,000
"""

naive_prompt_6 = "Suggest an investment portfolio for this client: {profile}"

execute_prompt(
    naive_prompt_6,
    "4.1 Naive Investment Suggestion",
    profile=client_profile_1
)

--- 4.1 Naive Investment Suggestion ---
PROMPT:
Human: Suggest an investment portfolio for this client: 
- Name: Sameer Khan
- Age: 28
- Goal: Wealth accumulation for early retirement
- Risk Tolerance: Moderate
- Annual Income: ₹25,00,000


==================== RESPONSE ====================

Creating a well-balanced investment portfolio for Sameer Khan, who is 28 years old, has a moderate risk tolerance, and aims for wealth accumulation for early retirement, involves a mix of asset classes. Below is a suggested investment portfolio that aligns with his goals and risk profile:

### Suggested Investment Portfolio

#### 1. **Equities (40%)**
   - **Large-Cap Mutual Funds (20%)**: Invest in established companies with a strong track record. Consider funds like HDFC Top 100 Fund or ICICI Prudential Bluechip Fund.
   - **Mid-Cap Mutual Funds (10%)**: These can provide higher growth potential. Look at funds like Axis Midcap Fund or Kotak Emerging Equity Fund.
   - **Small-Cap Mutual Funds (10%

#### **"ToT-style" Prompt 👍**

This prompt guides the model to explore and critique several strategies before making a final, well-justified recommendation.

In [ ]:
client_profile_1 = """
- Name: Sameer Khan
- Age: 28
- Goal: Wealth accumulation for early retirement
- Risk Tolerance: Moderate
- Annual Income: ₹25,00,000
"""

tot_prompt_1 = """
You are a senior investment strategist. Your task is to devise an investment strategy for a new client.
Use the following structured thinking process:

Client Profile:
{profile}

1.  **Propose Three Distinct Strategies:** Based on the client's profile, generate three different investment strategies. Name each one (e.g., 'Balanced Growth', 'Aggressive Tech Focus', 'Diversified Core'). For each, suggest a sample asset allocation (Equities, Debt, Alternatives).
2.  **Evaluate Each Strategy:** For each of the three strategies, briefly list the potential Pros and Cons specifically for this client. Consider their age, goal, and risk tolerance.
3.  **Recommend and Justify:** Based on your evaluation, select the single most suitable strategy for the client and write a concluding paragraph explaining why it is the best choice.
"""

execute_prompt(
    tot_prompt_1,
    "4.1 ToT-style Investment Strategy",
    profile=client_profile_1
)

--- 4.1 ToT-style Investment Strategy ---
PROMPT:
Human: 
You are a senior investment strategist. Your task is to devise an investment strategy for a new client.
Use the following structured thinking process:

Client Profile:

- Name: Sameer Khan
- Age: 28
- Goal: Wealth accumulation for early retirement
- Risk Tolerance: Moderate
- Annual Income: ₹25,00,000


1.  **Propose Three Distinct Strategies:** Based on the client's profile, generate three different investment strategies. Name each one (e.g., 'Balanced Growth', 'Aggressive Tech Focus', 'Diversified Core'). For each, suggest a sample asset allocation (Equities, Debt, Alternatives).
2.  **Evaluate Each Strategy:** For each of the three strategies, briefly list the potential Pros and Cons specifically for this client. Consider their age, goal, and risk tolerance.
3.  **Recommend and Justify:** Based on your evaluation, select the single most suitable strategy for the client and write a concluding paragraph explaining why it is the


-----

### **4.2. Tree-of-Thought (ToT): Devising a Debt Restructuring Plan**

**Scenario:** A financial advisor is helping a client who is overwhelmed with multiple sources of debt. The goal is to propose and compare different concrete strategies for tackling the debt.

In [ ]:
client_debt_profile = """
- Client: Priya
- Monthly Take-home Salary: ₹80,000
- Monthly Expenses (non-debt): ₹45,000
- Available for Debt Repayment: ₹35,000 per month
- Debts:
  1. Credit Card: ₹2,00,000 at 36% APR
  2. Personal Loan: ₹5,00,000 at 14% APR
  3. Car Loan: ₹3,00,000 at 9% APR
"""

#### **Naive Prompt 👎**

This prompt will generate generic, non-comparative advice that isn't tailored into actionable, alternative strategies.

In [ ]:
naive_prompt_4 = "How can this client manage their debt? \n\n{profile}"

execute_prompt(
    naive_prompt_4,
    "4.1 Naive Debt Advice",
    profile=client_debt_profile
)

--- 4.1 Naive Debt Advice ---
PROMPT:
Human: How can this client manage their debt? 


- Client: Priya
- Monthly Take-home Salary: ₹80,000
- Monthly Expenses (non-debt): ₹45,000
- Available for Debt Repayment: ₹35,000 per month
- Debts:
  1. Credit Card: ₹2,00,000 at 36% APR
  2. Personal Loan: ₹5,00,000 at 14% APR
  3. Car Loan: ₹3,00,000 at 9% APR


==================== RESPONSE ====================

Managing debt effectively requires a strategic approach, especially when dealing with multiple loans at varying interest rates. Here’s a step-by-step plan for Priya to manage her debt:

### 1. **Assess the Situation**
   - **Total Monthly Income**: ₹80,000
   - **Total Monthly Expenses**: ₹45,000
   - **Available for Debt Repayment**: ₹35,000

### 2. **Understand the Debt**
   - **Credit Card Debt**: ₹2,00,000 at 36% APR
   - **Personal Loan**: ₹5,00,000 at 14% APR
   - **Car Loan**: ₹3,00,000 at 9% APR

### 3. **Prioritize Debt Repayment**
   - **Focus on High-Interest Debt First**: The

*Critique:* The advice is sound but lacks structure and fails to explore or compare alternative, well-known strategies.

#### **Improved "ToT-style" Prompt 👍**

This prompt instructs the model to generate and evaluate multiple distinct strategies, presenting a comprehensive analysis of the options.

In [ ]:
tot_prompt_4 = """
You are an expert financial advisor. For the client profile below, devise a debt management plan using the following structured thinking process.

Client Profile:
{profile}

1.  **Propose Three Distinct Strategies:** Generate three different debt repayment strategies. Name each one (e.g., 'Debt Avalanche', 'Debt Snowball', 'Loan Consolidation'). Briefly explain how each would work for this client.
2.  **Evaluate Each Strategy:** For each of the three strategies, list the primary 'Pro' (e.g., lowest total interest paid) and the primary 'Con' (e.g., may feel slower) for this specific client.
3.  **Recommend and Justify:** Based on your evaluation, select the single most suitable strategy and explain why it is the best recommendation for someone in this situation.
"""

execute_prompt(
    tot_prompt_4,
    "4.2 ToT-style Debt Plan",
    profile=client_debt_profile
)

--- 4.2 ToT-style Debt Plan ---
PROMPT:
Human: 
You are an expert financial advisor. For the client profile below, devise a debt management plan using the following structured thinking process.

Client Profile:

- Client: Priya
- Monthly Take-home Salary: ₹80,000
- Monthly Expenses (non-debt): ₹45,000
- Available for Debt Repayment: ₹35,000 per month
- Debts:
  1. Credit Card: ₹2,00,000 at 36% APR
  2. Personal Loan: ₹5,00,000 at 14% APR
  3. Car Loan: ₹3,00,000 at 9% APR


1.  **Propose Three Distinct Strategies:** Generate three different debt repayment strategies. Name each one (e.g., 'Debt Avalanche', 'Debt Snowball', 'Loan Consolidation'). Briefly explain how each would work for this client.
2.  **Evaluate Each Strategy:** For each of the three strategies, list the primary 'Pro' (e.g., lowest total interest paid) and the primary 'Con' (e.g., may feel slower) for this specific client.
3.  **Recommend and Justify:** Based on your evaluation, select the single most suitable strategy 


*Critique:* This response provides a strategic, multi-faceted analysis that empowers the client and advisor to make an informed decision based on clear trade-offs.